<a href="https://colab.research.google.com/github/VirtualAkseli/IDV_22_IMF/blob/main/IDV_22_project_AIPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly
!pip install pandas
!pip install xlrd==1.2.0

In [2]:
import pandas as pd
import plotly as pt

In [ ]:
data = pd.read_csv('imf-dm.csv')

!pip install geopandas
import geopandas as gpd

In [4]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [5]:
cond_rus = world['name'] == 'Russia'
cond_ame = world['name'] == 'United States of America'
world.loc[cond_rus, 'name'] = 'Russian Federation'
world.loc[cond_ame, 'name'] = 'United States'

In [6]:
GDP_json = world[['name', 'geometry']]

In [7]:
combined = world.join(data.iloc[1:,:].set_index('Real GDP growth (Annual percent change)'), on='name')


In [ ]:
exploded = combined.explode()

In [ ]:
!pip install folium
import folium
import branca.colormap as cm

In [ ]:
x_map=combined.centroid.x.mean()
y_map=combined.centroid.y.mean()
print(x_map,y_map)

In [10]:
import numpy as np
years = []
for y in range(1980, 2027):
  years.append(str(y))
combined = combined.replace('no data', 0)
combined[years] = combined[years].astype(float)
combined.replace(0.0, np.nan, inplace=True)

In [ ]:
import numpy as np
minmaxlist = pd.DataFrame(np.linspace(-67.0, 125.0, 20))
myscale = (minmaxlist.quantile((0,0.1, 0.25, 0.5, 0.75,0.9,0.98,1))).to_numpy().ravel().tolist()
urs = ['name']
combined.info()

In [32]:
gdp_dict = combined.set_index('name')[years[0:]]


In [ ]:
#altered from https://github.com/python-visualization/folium/blob/main/examples/TimeSliderChoropleth.ipynb
n_periods = 47
datetime_index = pd.date_range(start=pd.datetime(1980, 1, 1), periods=n_periods, freq='AS')
dt_index_epochs = datetime_index.astype(int) // 10 ** 9
dt_index = dt_index_epochs.astype("U10")



In [40]:
#altered from https://github.com/python-visualization/folium/blob/main/examples/TimeSliderChoropleth.ipynb
n_sample = 20
styledata = {}

for country in combined.index:
    df = pd.DataFrame(
        {
            "color": np.random.normal(size=n_periods),
            "opacity": np.random.normal(size=n_periods),
        },
        index=dt_index,
    )
    df = df.cumsum()
    df.sample(n_sample, replace=False).sort_index()
    styledata[country] = df

In [41]:
max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

for country, data in styledata.items():
    max_color = max(max_color, data["color"].max())
    min_color = min(max_color, data["color"].min())
    max_opacity = max(max_color, data["opacity"].max())
    max_opacity = min(max_color, data["opacity"].max())

from branca.colormap import linear

cmap = linear.PuRd_09.scale(min_color, max_color)

def norm(x):
    return (x - x.min()) / (x.max() - x.min())


for country, data in styledata.items():
    data["color"] = data["color"].apply(cmap)
    data["opacity"] = norm(data["opacity"])

styledict = {
    str(country): data.to_dict(orient="index") for country, data in styledata.items()
}

In [42]:
from folium.plugins import TimeSliderChoropleth
mymap = folium.Map(location=[y_map, x_map], zoom_start=4)

g = TimeSliderChoropleth(
    GDP_json.to_json(),
    styledict=styledict, overlay = True
).add_to(mymap)

colmonos = years[:9]
'''
mymap.choropleth(
    geo_data=GDP_json,
    data=combined,
    columns=colmonos,
    key_on='feature.properties.name',
    fill_color= 'BuPu',
    threshold_scale=myscale,
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='GDP in %'
)


folium.Choropleth(
    geo_data=world,
    data=combined,
    columns=years,
    fill_color='YlGnBu',
    key_on='feature.id',
    bins=[-67, -10, 10, 125],
    legend_name='GDP in %',
    ).add_to(mymap)

mymap.choropleth(
 geo_data=combined['geometry'],
 name='Choropleth',
 data=combined,
 columns=colmonos,
 key_on="feature.properties.name",
 fill_color='YlGnBu',
 threshold_scale=myscale,
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='GDP in %',
 smooth_factor=0
)
)
'''
style_function = lambda x: {'fillColor': 'green',
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    combined,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(fields=['gdp_md_est','pop_est'],
            aliases=['GDP','Population estimate'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    ))


mymap.add_child(NIL)
folium.LayerControl().add_to(mymap)


mymap


In [31]:
!pip install mapclassify

In [45]:
from ipywidgets import interact, interactive, fixed, interact_manual
import mapclassify
import ipywidgets as widgets

def worldplot(date):
    combined.plot(column=date,cmap='RdYlGn',figsize=(24,12), legend = True, scheme="equal_interval", k=10, missing_kwds={

        "color": "lightgrey",

        "edgecolor": "red",

        "hatch": "///",

        "label": "Missing values",

    },)
ss = widgets.SelectionSlider(
    options=years,
    value='2000',
    description='Year of GDP change',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

interact(worldplot,date=ss)


interactive(children=(SelectionSlider(continuous_update=False, description='Year of GDP change', index=20, opt…

<function __main__.worldplot>